In [18]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [19]:
train = pd.read_csv('CS 583 Project/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('CS 583 Project/validation.csv')
test = pd.read_csv('CS 583 Project/test.csv')

In [20]:
train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train = train.loc[:12000, :]

In [21]:
xtrain, xvalid, ytrain, yvalid = train_test_split(
    train['comment_text'], train['toxic'], stratify=train['toxic'], random_state=42, test_size=0.2, shuffle=True
)

In [22]:
def load_glove_vectors(file_path):
    word_vectors = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

glove_vectors = load_glove_vectors('CS 583 Project/glove.6B.300d.txt')

In [23]:
def build_vocab(texts, max_vocab_size=20000):
    word_counter = Counter()
    for text in texts:
        word_counter.update(text.split())
    most_common = word_counter.most_common(max_vocab_size)
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(most_common)}
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1
    return vocab

vocab = build_vocab(xtrain)

In [24]:
embedding_dim = 300
embedding_matrix = np.zeros((len(vocab), embedding_dim))
for word, idx in vocab.items():
    if word in glove_vectors:
        embedding_matrix[idx] = glove_vectors[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))


In [28]:
def text_to_sequence(text, vocab, max_len=150):
    words = text.split()
    sequence = [vocab.get(word, vocab["<UNK>"]) for word in words[:max_len]]
    sequence += [vocab["<PAD>"]] * (max_len - len(sequence))
    return sequence

In [29]:
xtrain_seq = xtrain.apply(lambda x: text_to_sequence(x, vocab))
xvalid_seq = xvalid.apply(lambda x: text_to_sequence(x, vocab))

In [30]:
class ToxicCommentDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=150):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        sequence = text_to_sequence(text, self.vocab, self.max_len)
        return torch.tensor(sequence, dtype=torch.long), torch.tensor(label, dtype=torch.float)

In [31]:
train_dataset = ToxicCommentsDataset(xtrain_seq, ytrain)
valid_dataset = ToxicCommentsDataset(xvalid_seq, yvalid)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [32]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, pad_idx):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_out, _ = self.rnn(embedded)
        final_hidden_state = rnn_out[:, -1, :]  # Last hidden state
        output = self.fc(final_hidden_state)
        return self.sigmoid(output).squeeze()


In [33]:
model = ToxicCommentRNNClassifier(embedding_matrix)
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [40]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            targets = batch["target"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {total_loss / len(train_loader)}")

        # Validation
        model.eval()
        valid_preds = []
        valid_targets = []
        with torch.no_grad():
            for batch in valid_loader:
                input_ids = batch["input_ids"].to(device)
                targets = batch["target"].to(device)

                outputs = model(input_ids).squeeze()
                valid_preds.extend(outputs.cpu().numpy().flatten().tolist())  # Fix: Flatten and convert to list
                valid_targets.extend(targets.cpu().numpy().tolist())

        # Threshold for binary classification
        valid_preds_binary = np.array(valid_preds) > 0.5

        # Metrics
        auc = roc_auc_score(valid_targets, valid_preds)
        acc = accuracy_score(valid_targets, valid_preds_binary)
        precision = precision_score(valid_targets, valid_preds_binary, zero_division=1)
        recall = recall_score(valid_targets, valid_preds_binary)
        f1 = f1_score(valid_targets, valid_preds_binary)
        cm = confusion_matrix(valid_targets, valid_preds_binary)

        print(f"Validation AUC: {auc}")
        print(f"Validation Accuracy: {acc}")
        print(f"Validation Precision: {precision}")
        print(f"Validation Recall: {recall}")
        print(f"Validation F1-Score: {f1}")
        print(f"Confusion Matrix:\n{cm}")



train_model(model, train_loader, valid_loader, criterion, optimizer)

Epoch 1/3, Training Loss: 0.3059838726868232
Validation AUC: 0.5249616006549165
Validation Accuracy: 0.9058725531028738
Validation Precision: 1.0
Validation Recall: 0.004405286343612335
Validation F1-Score: 0.008771929824561403
Confusion Matrix:
[[2174    0]
 [ 226    1]]
Epoch 2/3, Training Loss: 0.3031545462956031
Validation AUC: 0.5363648890167741
Validation Accuracy: 0.9046230737192836
Validation Precision: 0.25
Validation Recall: 0.004405286343612335
Validation F1-Score: 0.008658008658008658
Confusion Matrix:
[[2171    3]
 [ 226    1]]
Epoch 3/3, Training Loss: 0.301729512338837
Validation AUC: 0.5178095959862046
Validation Accuracy: 0.9025406080799667
Validation Precision: 0.18181818181818182
Validation Recall: 0.00881057268722467
Validation F1-Score: 0.01680672268907563
Confusion Matrix:
[[2165    9]
 [ 225    2]]


In [42]:
def predict_toxicity(comment, model, vocab, max_len=150):
    model.eval()
    with torch.no_grad():
        # Convert comment to a sequence
        sequence = text_to_sequence(comment, vocab, max_len)
        input_ids = torch.tensor(sequence, dtype=torch.long).unsqueeze(0).to(device)

        # Predict the toxicity
        outputs = model(input_ids)
        prediction = outputs.cpu().numpy().flatten()

        # Convert predictions to binary (threshold = 0.5)
        prediction_binary = (prediction > 0.5).astype(int)

    # Display the result
    categories = ['toxic']  # Update the categories if you are predicting multiple labels
    for category, pred in zip(categories, prediction_binary):
        print(f"{category}: {'Yes' if pred == 1 else 'No'}")

# Test the function
test_comment = "You're such a horrible person!"
predict_toxicity(test_comment, model, vocab)

test_comment1 = "You should go to hell!"
predict_toxicity(test_comment1, model, vocab)

test_comment2 = "You are amazing!"
predict_toxicity(test_comment2, model, vocab)


toxic: No
toxic: No
toxic: No
